# <center>Introduction to viASP</center>


viASP is a visualization and interactive explanation tool for ASP programs and their stable models.

viASP allows you to explore the visualization in a variety of ways:

* Visualize the derivation of stable models step-by-step in a tree view
* Inspect iterations of recursive rules individually
* Show reasons for the derivation of individual symbols with arrows
* Relax the constraints of unsatisfiable programs
* Inspect single models
* Add `#show` statements on the fly
* Search models, signatures and rules.

![Example visualization](../docs/img/sprinkler.png)

To get started, the startup module starts the viASP server:

In [ ]:
colors = {
    # node border colors
    "ten": {"dark": "#3FA7D1"}, 
    # row background (arbitrary number and names)
    "twenty": { "dark": "#a9a9a94a", "bright": "#ffffff"},
    # text color of node, detail sidebar, row header (dark) and detail sidebar atoms (bright)
    "thirty": {"dark": "#444", "bright": "#F6F4F3"},
    # recursive node supernode background (dark) and border (bright)
    "fourty": { "0": "#3FA7D1", "1": "#3FA7D1"},
    # detail sidebar atom background (dark) and border (bright)
    "fifty": { "dark": "#3FA7D1"},
    # background color of node, detail sidebar, row header
    "sixty": {"dark": "#F6F4F3"},
    # edge color (dark) and edge to clingraph color (bright)
    "seventy": { "dark": "#000000", "bright": "#000000"},
    # arbitrary number of colors to highlight explanations
    "highlight": { "0": "#d48521", "1": "#9a8298", "2": "#e0e4ac", "3": "#98f4e2", "4": "#21d485"}, 
    # currently not used
    "error": {"ten": "#EB4A4E", "thirty": "#4C191A", "sixty": "#FCE8E8"},
    "warn": {"ten": "#FF9800", "thirty": "#653300", "sixty": "#FFF1DF"}}

In [ ]:
from viasp.server import startup
app = startup.run(colors=colors)
# if this cell encounters an error in binder, restart the kernel and run it again

Defining an encoding.

In [ ]:
%%file sprinkler.lp
1{rain;sprinkler}1.
wet :- rain.
wet :- sprinkler.

In [ ]:
%%file hamiltonian.lp

node(1..4). start(1).
edge(1,2). edge(2,3). edge(2,4). edge(3,1).
edge(3,4). edge(4,1). edge(4,3).

{ hc(V,U) } :- edge(V,U).
reached(V) :- hc(S,V), start(S).
reached(V) :- reached(U), hc(U,V).
:- node(V), not reached(V).
:- hc(V,U), hc(V,W), U!=W.
:- hc(U,V), hc(W,V), U!=W.
% :- hc(V,U), hc(W,V), U!=W. % uncomment to make the program unsatisfiable & comment the line above


In [ ]:
from viasp import Control

options = ['0']
programs = ['sprinkler.lp']
ctl = Control(options)
for path in programs:
    ctl.load(path)
ctl.ground([("base", [])])

with ctl.solve(yield_=True) as handle:
    for m in handle:
        print("Answer:\n{}".format(m))
        ctl.viasp.mark(m) # mark the answer set for visualization
    print(handle.get())
ctl.viasp.show() # start the graph generation
app.run()        # run the Dash app